In [565]:
import numpy as np
import wave

def read_wav_file(file_name,tsh):
    spf = wave.open(file_name, "r")

    # Extract Raw Audio from Wav File
    signal = spf.readframes(-1)
    signal = np.frombuffer(signal, np.int16)
    fs = spf.getframerate()

    signal = signal/np.max(signal)
    Time = np.linspace(0, len(signal) / fs, num=len(signal))


    signal = signal[10000:40000]
    Time = Time[10000:40000]

    mask = (  ((signal < tsh) & (signal > 0)) | (signal > -tsh) & (signal < 0))  
    filtered = signal.copy()
    filtered[mask] = 0

    return signal, Time, fs, filtered



tsh = 0.1
signal, Time, fs, filtered = read_wav_file("output_2.wav",tsh)



In [566]:
import plotly.express as px
fig = px.line( y=signal, width=4000, height=400 )
fig.show()

fig = px.line( y=filtered, width=4000, height=400 )
fig.show()

In [563]:

def get_second_peak_position(signal):
    def get_second_peak(filtered):
        maxa = 0
        max2 = 0
        for i,x in enumerate(filtered):
            if x < 0.1:
                continue
            
            else:
                maxa = max(maxa, x)

            if x < maxa:
                break

        for j,x in enumerate(filtered[i:]):
            if x < maxa:
                continue
            else:
                max2 = max(max2, x)

            if x < max2:
                break
        return max2, j+i-1


    maxa, la = get_second_peak(signal)
    maxb, lb = get_second_peak(signal[20000:])
    lb += 20000

    return la, lb, maxa, maxb


la, lb, maxa, maxb = get_second_peak_position(filtered)
print("The first peak is at:", la, maxa)
print("The second peak is at:", lb, maxb)

delay_1 = (Time[lb] - Time[la])*1000

print("The distance between the two peaks is:", round(delay_1,2)    )

The first peak is at: 5053 0.32967849842036795
The second peak is at: 22334 1.0
The distance between the two peaks is: 391.87


In [564]:
# import numpy as np
# import matplotlib.pyplot as plt
# from scipy.signal import butter, filtfilt

# def butter_bandpass(lowcut, highcut, fs, order=5):
#    nyq = 0.5 * fs
#    low = lowcut / nyq
#    high = highcut / nyq
#    b, a = butter(order, [low, high], btype='band')
#    return b, a

# def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
#    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
#    y = filtfilt(b, a, data)
#    return y

# fs = fs 
# t = Time
# f1 = 900  
# f2 = 1100  
# sig = signal
# order = 4  
# filtered = butter_bandpass_filter(sig, f1, f2, fs, order)
# print("The length of frequencies:",len(filtered))
# # print("The Butterworth bandpass filter output of first 50 frequencies:",filtered[:50])

